In [ ]:
! pip install pdfminer.six
! pip install doc2text
! pip install nltk
! pip install spacy
! python -m spacy download en_core_web_sm
! pip install nltk
! python -m nltk nltk.download('words')
! pip install constants

In [209]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from nltk.corpus import stopwords
import nltk
import constants as cs
import io
import json
import re
import spacy
import pandas as pd
from spacy.matcher import Matcher
from nltk.stem import WordNetLemmatizer


# enc_core_lg

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
            yield text
            converter.close()
            fake_file_handle.close()


def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text


def extract_text(pdf_path):
    resume_text = ''
    for page in extract_text_from_pdf(pdf_path):
        resume_text += page
    return resume_text


# def extract_education(resume_text):
#     STOPWORDS = set(stopwords.words('english'))
#     EDUCATION = ['BE','B.E.', 'B.E', 'BS','B.S','B.Com','BCA','ME','M.E', 'M.E.', 'M.S','B.com','10','10+2','BTECH', 'B.TECH', 'M.TECH', 'MTECH', 'SSC', 'HSC', 'C.B.S.E','CBSE','ICSE', 'X', 'XII','10th','12th',' 10th',' 12th','Bachelor of Arts in Mathematics','Master of Science in Analytics','Bachelor of Business Administration','Major: Business Management']

#     nlp_text = nlp(resume_text)
#     nlp_text = [sent.text.strip() for sent in nlp_text.sents]

#     edu = {}
#     for index, text in enumerate(nlp_text):
#         for tex in text.split():
#             tex = re.sub(r'[?|$|.|!|,]', r'', tex)
#             if tex.upper() in EDUCATION and tex not in STOPWORDS:
#                 edu[tex] = text + nlp_text[index + 1]

#     education = []
#     for key in edu.keys():
#         year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
#         if year:
#             education.append((key, ''.join(year[0])))
#         else:
#             education.append(key)
#     return education


def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    # regex working with +14155827457,+91 9223339060, 614-555-5555, (520) 271-2492, +919619496873 , +91-9619496873  
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number


def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

# def extract_skills(resume_text):
# nlp_text = nlp(resume_text)
# noun_chunks = nlp_text.noun_chunks

# def extract_skills(resume_text):
#     nlp = spacy.load('en_core_web_sm')
#     nlp_text = nlp(resume_text)
#     noun_chunks = nlp_text.noun_chunks
    
#     # tokens = [token.text for token in nlp_text if not token.is_stop]

#     tokens = [token.text for token in nlp_text if token.is_stop]

#     data = pd.read_csv("skills.csv") 
#     skills = list(data.columns.values)
    
#     skillset = []
#     for token in tokens:
#         if token.lower() in skills:
#             skillset.append(token)
 
#     for token in noun_chunks:
#         token = token.text.lower().strip()
#         if token in skills:
#             skillset.append(token)
    
#     return [i.capitalize() for i in set([i.lower() for i in skillset])]

def extract_dob(text):
    result1=re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}",text) # 31/12/2001
    result2=re.findall(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}",text) # 31-12-2001
    #TODO: result3 nd, st, rd, th
    result3= re.findall(r"[\d]{1,2} [ADFJMNOSadfjmnos]\w* [\d]{4}",text) # 31 December 2001
    result4=re.findall(r"([\d]{1,2})\.([\d]{1,2})\.([\d]{4})",text) # 31.12.2001
                
    l=[result1,result2,result3,result4]
    for i in l:
        if i==[]:
            continue
        else:
            return i

# def get_experience(document):
#     pattern1 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4})')
#     pattern2 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(\d{2}(.|..)\d{4})')
#     pattern3 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(present)')
#     pattern4 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(present)')
#     patterns = [pattern1, pattern2, pattern3, pattern4]
#     experience = []
#     for index, line in enumerate(document):
#         for pattern in patterns:
#             exp = pattern.findall(line)
#             if len(exp) > 0:
#                 experience.append(document[index:index+4])
#     return (experience)
    
def extract_location(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ['GPE', 'LOC']:
            ls = list(ent.text)
    string = ''.join(ls)
    return string

In [183]:
from datetime import datetime
from dateutil import relativedelta
import constants as cs

In [184]:
def extract_entity_sections_grad(text):
    text_split = [i.strip() for i in text.split('\n')]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(cs.RESUME_SECTIONS_GRAD)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in cs.RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)
    return entities

In [193]:
file_path = "D:\Coding --python\Resumes\SGresume-1.pdf"
text = ''
page = ''
for page in extract_text_from_pdf(file_path):
    text += ' ' + page

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

In [194]:
dictionary ={
    "Name" : extract_name(text),
    "Mobile Number" : extract_mobile_number(text),
    "Email" : extract_email(text),
    # "Address" : extract_address(text),
    "Date of Birth" : extract_dob(text),
    # "Location" : extract_location(text),
    "Entities" : extract_entity_sections_grad(text)
}

In [195]:
keyslist = list(dictionary["Entities"].keys())
keyslist

['experience', 'projects', 'education', 'leadership', 'skills', 'interests']

In [196]:
expereicelist = dictionary["Entities"]["experience"]
skillslist = dictionary["Entities"]["skills"]
projectlist = dictionary["Entities"]["projects"]
educationlist = dictionary["Entities"]["education"]

In [197]:
# deleting entities from dictionary
del dictionary["Entities"]

In [198]:
dict1 = {
    "experience" : expereicelist,
    "skills" : skillslist,
    "projects" : projectlist,
    "education" : educationlist
}

In [199]:
z = dictionary | dict1

In [200]:
json_object = json.dumps(z, indent = 4, ensure_ascii=False)
with open("extracted_details1.json", "w") as outfile:
    outfile.write(json_object)
# print(extract_name(text))
# print(extract_mobile_number(text))
# print(extract_email(text))
# print(extract_dob(text))
# print(extract_skills(text))
# print(extract_experience(text))